In [ ]:
import pandas as pd
df=pd.read_csv('complaints.csv')

C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_18240\2474268451.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('complaints.csv')


In [3]:
df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2025-06-13,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,"EQUIFAX, INC.",NY,11433,NaN,NaN,Web,2025-06-13,In progress,Yes,NaN,14072365
1,2025-06-13,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",NJ,07047,NaN,NaN,Web,2025-06-13,In progress,Yes,NaN,14072555
2,2025-06-13,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",NH,03051,NaN,NaN,Web,2025-06-13,In progress,Yes,NaN,14072377
3,2025-06-13,Credit reporting or other personal consumer re...,Credit reporting,Problem with a company's investigation into an...,Investigation took more than 30 days,NaN,NaN,"EQUIFAX, INC.",CA,94080,NaN,NaN,Web,2025-06-13,In progress,Yes,NaN,14072294
4,2025-06-13,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",CA,92583,NaN,NaN,Web,2025-06-13,In progress,Yes,NaN,14072340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9589245,2016-09-08,Credit reporting,NaN,Unable to get credit report/credit score,Problem getting my free annual report,NaN,NaN,"EQUIFAX, INC.",FL,32853,NaN,Consent not provided,Web,2016-09-08,Closed with explanation,Yes,Yes,2102374
9589246,2022-06-06,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,NaN,"EQUIFAX, INC.",CA,93638,NaN,Consent not provided,Web,2022-06-06,Closed with explanation,Yes,NaN,5637858
9589247,2016-07-14,Credit reporting,NaN,Unable to get credit report/credit score,Problem getting report or credit score,NaN,NaN,"EQUIFAX, INC.",NJ,08620,"Older American, Servicemember",NaN,Phone,2016-07-14,Closed with explanation,Yes,No,2012835
9589248,2014-03-03,Credit reporting,NaN,Improper use of my credit report,Report improperly shared by CRC,NaN,NaN,"EQUIFAX, INC.",NJ,08759,NaN,NaN,Web,2014-03-04,Closed with explanation,Yes,No,740118


In [4]:
df=df[['Product','Consumer complaint narrative']].dropna()
df = df[df['Product'].isin(df['Product'].value_counts().nlargest(5).index)]

In [5]:
#Reset index
df.reset_index(drop=True, inplace=True)

#### Data Cleaning and Preprocessing

In [6]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [11]:
from tqdm import tqdm
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
corpus = []

print("Preprocessing complaints...")

for i in tqdm(range(0, len(df))):
    review = re.sub('[^a-zA-Z]', ' ', df['Consumer complaint narrative'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stop_words]
    review = ' '.join(review)
    corpus.append(review)

Preprocessing complaints...


100%|██████████| 2498054/2498054 [56:27<00:00, 737.41it/s] 


In [15]:
y = pd.factorize(df['Product'])[0]

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.2, random_state=42)

In [25]:
#BOW
from sklearn.feature_extraction.text import CountVectorizer

# If X_train and X_test are already sparse matrices, skip this step.
# Otherwise, use the original text lists for vectorization.
if not hasattr(X_train, 'shape'):
	cv = CountVectorizer(max_features=2000, ngram_range=(1, 2))
	X_train = cv.fit_transform(X_train)  # X_train is a list of strings here
	X_test = cv.transform(X_test)        # X_test is a list of strings here

In [32]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))
X_train

<1998443x3000 sparse matrix of type '<class 'numpy.int64'>'
	with 133531158 stored elements in Compressed Sparse Row format>

In [26]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [36]:
# Step 1: Split raw text
X_train_raw, X_test_raw, y_train, y_test = train_test_split(corpus, y, test_size=0.2, random_state=42)

# Step 2: Vectorize
cv = CountVectorizer(max_features=2000, ngram_range=(1, 2))
X_train = cv.fit_transform(X_train_raw)   # returns sparse matrix
X_test = cv.transform(X_test_raw)

# Step 3: Train the model
model = MultinomialNB()
model.fit(X_train, y_train)

# Step 4: Predict
y_pred = model.predict(X_test)

In [38]:
from sklearn.metrics import classification_report, accuracy_score
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.6638584818989174

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.66      0.71    216506
           1       0.50      0.68      0.58     66763
           2       0.60      0.94      0.73     26326
           3       0.74      0.95      0.83     28096
           4       0.63      0.56      0.59    161920

    accuracy                           0.66    499611
   macro avg       0.65      0.76      0.69    499611
weighted avg       0.68      0.66      0.66    499611



In [39]:
print("\nLabel Mapping:")
for i, label in enumerate(pd.factorize(df['Product'])[1]):
    print(f"{i}: {label}")


Label Mapping:
0: Credit reporting or other personal consumer reports
1: Debt collection
2: Mortgage
3: Checking or savings account
4: Credit reporting, credit repair services, or other personal consumer reports


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(X_train_raw)
X_test_tfidf = tfidf.transform(X_test_raw)

In [41]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=200)
lr_model.fit(X_train_tfidf, y_train)
y_pred_lr = lr_model.predict(X_test_tfidf)


c:\PERSONAL\Conda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
from sklearn.metrics import classification_report, accuracy_score
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

Accuracy: 0.7894902233938004

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.81      0.81    216506
           1       0.79      0.74      0.77     66763
           2       0.90      0.90      0.90     26326
           3       0.92      0.93      0.93     28096
           4       0.72      0.73      0.73    161920

    accuracy                           0.79    499611
   macro avg       0.83      0.82      0.83    499611
weighted avg       0.79      0.79      0.79    499611



In [46]:
df.head()

,Product,Consumer complaint narrative
0,Credit reporting or other personal consumer re...,I am writing to have the following information...
1,Credit reporting or other personal consumer re...,"Upon reviewing my credit report, I have identi..."
2,Credit reporting or other personal consumer re...,I am filing a formal dispute regarding multipl...
3,Credit reporting or other personal consumer re...,This CFPB complaint has been filed to request ...
4,Credit reporting or other personal consumer re...,I am demanding immediate clarification regardi...


In [56]:
import pickle
pickle.dump(lr_model, open('model.pkl', 'wb'))
pickle.dump(tfidf, open('vectorizer.pkl', 'wb'))

# Save label mapping from pd.factorize
label_encoder = list(pd.factorize(df['Product'])[1])
pickle.dump(label_encoder, open('label_encoder.pkl', 'wb'))
